### Test ConsLinUCB, LinUCB, ThresholdBandit and GreedyBandit

In [1]:
%cd /home/lansdell/projects/conservativerdd
%pylab inline
pylab.rcParams['figure.figsize'] = (6, 6)

from lib.bandits import LinUCB, ThresholdBandit,ThresholdConsBandit,GreedyBandit, ConsLinUCB, expected_regret, expected_regret_per_arm
from lib.generator import LinearGeneratorParams, LinearGenerator
import numpy as np 

import matplotlib.pyplot as plt
from scipy.stats import truncnorm
import seaborn as sns
import pandas as pd
sns.set_style('ticks')

/home/lansdell/projects/conservativerdd
Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)
/home/lansdell/.local/lib/python2.7/site-packages/matplotlib/__init__.py:913: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
#alg = 'greedy'
#alg = 'linucb'
#alg = 'threshold'
#alg = 'thresholdcons'
alg = 'conslinucb'

M = 3    #number of runs
N = 10000 #number of timesteps
save = False
alpha = 0.1 #CLUCB parameter...
#alpha = 1.0 #CLUCB parameter...
max_alpha = 2
max_beta = 2
k = 4    #Number of arms
d = 5   #Dimension of context (includes one dim for intercept term)
intercept = True
evaluate_every = 100

if alg == 'greedy':
	BanditAlg = GreedyBandit
elif alg == 'linucb':
	BanditAlg = LinUCB
elif alg == 'threshold':
	BanditAlg = ThresholdBandit
elif alg == 'thresholdcons':
	BanditAlg = ThresholdConsBandit
elif alg == 'conslinucb':
	BanditAlg = ConsLinUCB
else:
	print("Select a valid algorithm")
    
baseline_idx = 1

In [3]:
def second_largest_idx(a):
    vals = a.sort()

## Run CLUCB 

In [4]:
#Generate slopes and intercepts
alphas = truncnorm.rvs(-max_alpha, max_alpha, scale = 1, size=(M,k,d-1))
betas = truncnorm.rvs(-max_beta, max_beta, scale = 1, size=(M,k))

#baseline_alphas = truncnorm.rvs(-max_alpha, max_alpha, scale = 1, size=(M,1,d-1))
#baseline_betas = truncnorm.rvs(-max_beta, max_beta, scale = 1, size=(M,1))

regret = np.zeros((M, N))
expt_regret = np.zeros((M, N))
arm_pulls = np.zeros((M, N, k+1))
n_changes = np.zeros((M, N))
update_pol = np.zeros((M, N))

In [5]:
print("Running %s algorithm"%alg)
expt_rewards_clucb = np.zeros((M, N))
expt_rewards_baseline_clucb = np.zeros((M, N))
BanditAlg = ConsLinUCB

for m in range(M):
    params = LinearGeneratorParams(np.atleast_2d(alphas[m,:,:]), betas[m,:], d = d, k = k, intercept = intercept)
    generator = LinearGenerator(params)
    means = expected_regret_per_arm(generator)
    #Choose the baseline arm as the 2nd best arm on average....
    print(means)
    sorted_means = np.sort(means)
    #m_idx = np.argmax(means)
    m_idx = np.where(means == sorted_means[baseline_idx])[0][0]
    print('Baseline arm is %d'%m_idx)
    base_alpha = alphas[m,m_idx,:]
    base_beta = betas[m,m_idx]
    bandit = BanditAlg(generator, (base_alpha, base_beta), alpha = alpha)
    print("Run: %d/%d"%(m+1,M))
    for i in range(N):
        (ctx, arm_idx, obs, r) = bandit.step()
        regret[m,i] = r
        print(i, arm_idx, r)
        if arm_idx >= 0:
            arm_pulls[m,i,arm_idx] = 1
            expt_rewards_clucb[m,i] = np.dot(ctx[1:], alphas[m,arm_idx,:]) + betas[m,arm_idx]
        else:
            arm_pulls[m,i,m_idx] = 1
            expt_rewards_clucb[m,i] = np.dot(ctx[1:], alphas[m,m_idx,:]) + betas[m,m_idx]
        if arm_idx == m_idx:
            print("idx: %d -- playing baseline arm"%i)
        expt_rewards_baseline_clucb[m,i] = np.dot(ctx[1:],base_alpha) + base_beta


Running conslinucb algorithm
[  1.52850976e+00   1.74727637e+00   3.58826941e+00   1.34815749e-04]
Baseline arm is 0
Run: 1/3
(0, -1, 1.5688393467721324)
(1, -1, 1.7012560783235851)
(2, 0, 1.4148618711738257)
idx: 2 -- playing baseline arm
(3, -1, 2.403896522544271)
(4, -1, 0.76895873169096984)
(5, -1, 1.2185099609888099)
(6, -1, 2.1725547695580598)
(7, -1, 1.901833718661778)
(8, -1, 1.378610159656485)
(9, -1, 0.84890994569223288)
(10, -1, 0.48870653433689915)
(11, -1, 2.4331535386923822)
(12, -1, 1.3398952009302114)
(13, -1, 0.935832266856647)
(14, -1, 1.9463007459535171)
(15, -1, 0.2594461050027993)
(16, -1, 0.97659572847497977)
(17, -1, 1.7558655683047675)
(18, -1, 2.6648011583884648)
(19, -1, 2.1883685040869123)
(20, -1, 2.8488450174333591)
(21, -1, 2.0361931699167686)
(22, -1, 1.7647269172634807)
(23, -1, 0.31124727466904301)
(24, -1, 1.0650804184057265)
(25, -1, 1.8881135271783944)
(26, -1, 0.4064283869576375)
(27, -1, 0.6515266484454455)
(28, -1, 0.77449235798572258)
(29, -1, 0.

(357, -1, 2.1262811686958956)
(358, -1, 2.5892899316896987)
(359, -1, 1.9301487100756916)
(360, -1, 1.8113506502383545)
(361, -1, 0.85649979805286369)
(362, -1, 2.090423503569292)
(363, -1, 1.8853171537612092)
(364, -1, 1.7227019213229855)
(365, -1, 1.8833624233721702)
(366, -1, 0.89493366918229211)
(367, -1, 0.4459724866088659)
(368, -1, 0.35755989452487491)
(369, -1, 2.4526575364914489)
(370, -1, 0.98062524000919971)
(371, -1, 1.3915398292578189)
(372, -1, 0.50305292223242382)
(373, -1, 0.75268085732058743)
(374, -1, 2.2413144239634475)
(375, -1, 1.7518293481596241)
(376, -1, 0.70819589184651377)
(377, -1, 1.5378582932987885)
(378, -1, 1.8654266056740889)
(379, -1, 1.6368218046372254)
(380, -1, 2.2576449426574245)
(381, -1, 1.0397043405294699)
(382, -1, 2.0928691071559697)
(383, -1, 1.0982623332045911)
(384, -1, 2.0997358077390138)
(385, -1, 0.96065011786809118)
(386, -1, 1.5732494800970538)
(387, -1, 1.8423132579781938)
(388, -1, 1.3377019352446349)
(389, -1, 0.7468026002719953)
(39

(688, -1, 1.0626995656513247)
(689, -1, 1.0009546130778817)
(690, -1, 2.3585943850968887)
(691, -1, 0.92752479588103731)
(692, -1, 0.54512777769060272)
(693, -1, 1.5893564860169747)
(694, -1, 0.64628242145625925)
(695, -1, 0.61165080607087008)
(696, -1, 2.0534007334423787)
(697, -1, 2.1157796658676942)
(698, -1, 1.7355636640573386)
(699, -1, 1.0818939539100798)
(700, -1, 1.0819166837538681)
(701, -1, 0.48369916032647398)
(702, -1, 1.4384225218159252)
(703, -1, 2.1674777045784923)
(704, -1, 0.96679338240033241)
(705, -1, 2.4562058446379638)
(706, -1, 1.5541137159777516)
(707, -1, 1.0811102222480522)
(708, -1, 2.5002705241725303)
(709, -1, 2.1366489975444249)
(710, -1, 2.2834391354228343)
(711, -1, 1.6695587273815267)
(712, -1, 2.3743841183510197)
(713, -1, 0.78380053315874387)
(714, -1, 2.0962261666977238)
(715, -1, 1.4650540860703458)
(716, -1, 2.1802148511575776)
(717, -1, 2.0573575229742991)
(718, -1, 2.0255154198080234)
(719, -1, 1.8345546181261039)
(720, -1, 2.0922370128649908)
(72

(970, -1, 2.1480262901828051)
(971, -1, 0.73656124813114532)
(972, -1, 0.65199019511926759)
(973, -1, 1.0530251861922588)
(974, -1, 0.77815822047577066)
(975, -1, 1.3677268533334077)
(976, -1, 1.8017797110581877)
(977, -1, 0.89898995118154312)
(978, -1, 2.3034081383336513)
(979, -1, 1.3196894198658944)
(980, -1, 0.4495439179870272)
(981, -1, 1.2277532383658485)
(982, -1, 1.8726752271176457)
(983, -1, 1.649738450064071)
(984, -1, 1.3672535189137065)
(985, -1, 2.143178845504738)
(986, -1, 0.88983737274353836)
(987, -1, 1.5492092418342853)
(988, -1, 2.1087947292735447)
(989, -1, 2.2461074835091628)
(990, -1, 1.9873297813097481)
(991, -1, 0.34730674063984579)
(992, -1, 1.7290168696914545)
(993, -1, 1.531836786471142)
(994, -1, 1.2014440023486461)
(995, -1, 0.87416571825546185)
(996, -1, 0.80178418023777565)
(997, -1, 2.4158745349841158)
(998, -1, 1.1485639056932184)
(999, -1, 1.8206766564570507)
(1000, -1, 0.74068602914772874)
(1001, -1, 1.3263336365558596)
(1002, -1, 1.4006256503933299)
(

(1318, -1, 1.1709383859674314)
(1319, -1, 2.4155722080276107)
(1320, -1, 1.3259724196421208)
(1321, -1, 1.6543485348465869)
(1322, -1, 0.93366580058617066)
(1323, -1, 0.94039678003303395)
(1324, -1, 1.7982594061242028)
(1325, -1, 0.77641168011473294)
(1326, -1, 2.3899938534640608)
(1327, -1, 2.3794371233818348)
(1328, -1, 1.5464865388626956)
(1329, -1, 2.1141134485363682)
(1330, -1, 1.6616045182478498)
(1331, -1, 0.79577946567215474)
(1332, -1, 2.3104791185148099)
(1333, -1, 2.3188786439919915)
(1334, -1, 2.2842165427385597)
(1335, -1, 0.97312159137619214)
(1336, -1, 1.0968713093129778)
(1337, -1, 1.2411681001835966)
(1338, -1, 1.4783577265399206)
(1339, -1, 2.0055525274734798)
(1340, -1, 1.8159496936362596)
(1341, -1, 2.7210679139048555)
(1342, -1, 1.399167513176844)
(1343, -1, 2.2760877208730088)
(1344, -1, 2.0480076461633132)
(1345, -1, 1.4667875000826731)
(1346, -1, 1.3736370026574085)
(1347, -1, 1.7382842497733113)
(1348, -1, 2.083742237497836)
(1349, -1, 1.8677138840869745)
(1350

...

(8074, -1, 0.11288613648279489)
(8075, -1, 0.49343351002379432)
(8076, -1, 0.51537028090952708)
(8077, -1, 0.82929942218603769)
(8078, -1, 0.0)
(8079, -1, 0.71491178975779057)
(8080, -1, 0.94892335028954167)
(8081, -1, 0.0)
(8082, -1, 0.66228951960473803)
(8083, -1, 0.50658659392741034)
(8084, -1, 0.0)
(8085, -1, 0.83316209266835428)
(8086, -1, 0.82196418080791189)
(8087, -1, 0.25142845013902571)
(8088, -1, 0.47506491273934803)
(8089, -1, 0.0)
(8090, -1, 0.038388860436530114)
(8091, -1, 0.0)
(8092, -1, 0.59362232970543538)
(8093, -1, 0.31167557384454891)
(8094, -1, 0.0)
(8095, -1, 0.63474760561783072)
(8096, 3, 0.35397979194461726)
(8097, -1, 0.36892686515660489)
(8098, -1, 0.0)
(8099, -1, 0.0)
(8100, -1, 0.86635393785665071)
(8101, -1, 0.15878015930890066)
(8102, -1, 0.0)
(8103, -1, 0.0)
(8104, -1, 0.0)
(8105, -1, 0.3306034726475553)
(8106, -1, 0.0)
(8107, -1, 0.0)
(8108, -1, 0.68934507271056322)
(8109, -1, 0.28969633253163174)
(8110, -1, 0.85241049703149818)
(8111, -1, 0.0)
(8112, -1

(8458, -1, 0.0)
(8459, -1, 0.39438579969439025)
(8460, -1, 0.012479167639974587)
(8461, 0, 0.39913850436593412)
idx: 8461 -- playing baseline arm
(8462, 0, 0.0)
idx: 8462 -- playing baseline arm
(8463, -1, 0.454325626800687)
(8464, -1, 0.0)
(8465, -1, 0.67858818869773352)
(8466, 0, 0.0)
idx: 8466 -- playing baseline arm
(8467, -1, 0.75243883395776145)
(8468, -1, 0.0)
(8469, -1, 0.0)
(8470, 0, 0.0)
idx: 8470 -- playing baseline arm
(8471, -1, 0.052522052604322411)
(8472, -1, 0.0)
(8473, -1, 0.033006154136719035)
(8474, -1, 0.69665520713357254)
(8475, -1, 0.029940589616029967)
(8476, -1, 0.0)
(8477, -1, 0.0)
(8478, -1, 0.54112330441681511)
(8479, -1, 0.54469937741889962)
(8480, -1, 0.60592332710135854)
(8481, -1, 0.12317026476399895)
(8482, -1, 0.27565457261391213)
(8483, -1, 0.0)
(8484, -1, 0.21750446003069279)
(8485, -1, 0.0)
(8486, -1, 0.28566771070211283)
(8487, 0, 0.1697360892545251)
idx: 8487 -- playing baseline arm
(8488, -1, 0.18638667822171895)
(8489, -1, 0.11684644657393006)
(8

(8847, -1, 0.28999294839584988)
(8848, -1, 0.60879907020305146)
(8849, -1, 0.85178802741577098)
(8850, -1, 0.22454190742231983)
(8851, -1, 0.0)
(8852, -1, 0.45892642715052556)
(8853, 2, 1.7222972459183334)
(8854, -1, 0.0)
(8855, -1, 0.50095461143961795)
(8856, -1, 0.0)
(8857, -1, 1.0870028495977102)
(8858, -1, 0.0)
(8859, -1, 0.3664311142429173)
(8860, -1, 0.39380257294581211)
(8861, -1, 0.49073582247164882)
(8862, -1, 0.19335737996388291)
(8863, -1, 0.27660485908213328)
(8864, -1, 0.30976606053618838)
(8865, -1, 0.21012893679319533)
(8866, -1, 0.073036958976542943)
(8867, 0, 0.0)
idx: 8867 -- playing baseline arm
(8868, -1, 0.060719432517164273)
(8869, -1, 1.1466130367565124)
(8870, -1, 0.0)
(8871, -1, 0.0)
(8872, -1, 0.0)
(8873, -1, 0.080902776163045198)
(8874, -1, 0.57348107765535539)
(8875, -1, 1.0815155042337445)
(8876, 1, 0.0)
(8877, -1, 0.0)
(8878, -1, 0.48721903746615491)
(8879, -1, 0.27373960341241865)
(8880, -1, 0.0)
(8881, -1, 0.36213500981072833)
(8882, -1, 0.30245934921342

(9238, -1, 0.48518748833468772)
(9239, -1, 0.34228318347622899)
(9240, -1, 0.037958714132335647)
(9241, -1, 0.0)
(9242, -1, 0.2612505292509173)
(9243, -1, 0.086860642900602247)
(9244, -1, 0.66159691889360095)
(9245, -1, 0.0)
(9246, -1, 0.1631456328696852)
(9247, -1, 0.0)
(9248, -1, 0.21926405984305664)
(9249, -1, 0.0)
(9250, -1, 0.78110594643402376)
(9251, -1, 0.289583675633038)
(9252, -1, 0.3171563990320998)
(9253, -1, 0.31472078766661826)
(9254, -1, 0.0)
(9255, -1, 0.23489241583870424)
(9256, -1, 0.85826887236961569)
(9257, -1, 0.0)
(9258, -1, 0.82799431351316266)
(9259, -1, 0.73141897440075554)
(9260, 3, 0.44131927950130012)
(9261, -1, 0.0)
(9262, -1, 0.0)
(9263, -1, 0.23813770978688242)
(9264, -1, 0.0)
(9265, -1, 0.0)
(9266, -1, 0.45675295867978183)
(9267, -1, 0.0)
(9268, -1, 0.29315375964317592)
(9269, -1, 0.98519623090092456)
(9270, -1, 0.80721537544689514)
(9271, -1, 0.17054024834253045)
(9272, -1, 0.54322741333347047)
(9273, -1, 0.0)
(9274, -1, 0.0)
(9275, -1, 0.0)
(9276, 0, 0.

(9640, 0, 0.062577092820835345)
idx: 9640 -- playing baseline arm
(9641, -1, 0.65318180809267146)
(9642, -1, 0.6117983263268697)
(9643, -1, 0.0)
(9644, 0, 0.12736067725425504)
idx: 9644 -- playing baseline arm
(9645, -1, 0.0)
(9646, -1, 0.12699188473933165)
(9647, -1, 0.0)
(9648, -1, 0.0)
(9649, -1, 0.0)
(9650, -1, 0.88469260354872215)
(9651, -1, 0.0)
(9652, -1, 0.0)
(9653, -1, 0.59372675994943347)
(9654, -1, 0.44389668224036716)
(9655, -1, 0.14909783693294054)
(9656, -1, 0.0)
(9657, 0, 0.0)
idx: 9657 -- playing baseline arm
(9658, -1, 0.0)
(9659, -1, 0.0)
(9660, -1, 0.0)
(9661, -1, 0.3659962110206415)
(9662, 2, 0.029399444027449873)
(9663, -1, 0.35900173252806677)
(9664, -1, 1.1336981514070787)
(9665, -1, 0.42478902892016279)
(9666, -1, 0.95649229289872129)
(9667, -1, 0.0)
(9668, -1, 0.0)
(9669, -1, 0.77838935493740569)
(9670, -1, 0.0)
(9671, -1, 1.1809470905544768)
(9672, -1, 0.051571860802619329)
(9673, -1, 0.30753873694094869)
(9674, -1, 1.1046170188691531)
(9675, 0, 0.000710148057

## LinUCB for comparison

In [6]:
BanditAlg = LinUCB    
regret_linucb = np.zeros((M, N))
arm_pulls_linucb = np.zeros((M, N, k+1))
expt_rewards_linucb = np.zeros((M, N))
expt_rewards_baseline_linucb = np.zeros((M, N))

In [7]:
for m in range(M):
    params = LinearGeneratorParams(np.atleast_2d(alphas[m,:,:]), betas[m,:], d = d, k = k, intercept = intercept)
    generator = LinearGenerator(params)
    means = expected_regret_per_arm(generator)
    #Choose the baseline arm as the worst arm....
    sorted_means = np.sort(means)
    #m_idx = np.argmax(means)
    m_idx = np.where(means == sorted_means[baseline_idx])[0][0]
    base_alpha = alphas[m,m_idx,:]
    base_beta = betas[m,m_idx]
    bandit = BanditAlg(generator)
    print("Run: %d/%d"%(m+1,M))
    print('Worst arm is %d'%m_idx)
    for i in range(N):
        (ctx, arm_idx, obs, r) = bandit.step()
        regret_linucb[m,i] = r
        print(i, arm_idx, r)
        if arm_idx >= 0:
            arm_pulls_linucb[m,i,arm_idx] = 1
        else:
            arm_pulls_linucb[m,i,k] = 1
        if arm_idx == m_idx:
            print("idx: %d -- playing baseline arm"%i)
        expt_rewards_baseline_linucb[m,i] = np.dot(ctx[1:],base_alpha) + base_beta
        expt_rewards_linucb[m,i] = np.dot(ctx[1:], alphas[m,arm_idx,:]) + betas[m,arm_idx]


Run: 1/3
Worst arm is 0
(0, 0, 1.2494151248377816)
idx: 0 -- playing baseline arm
(1, 1, 2.201448216685383)
(2, 2, 4.0318311914393323)
(3, 3, 0.0)
(4, 0, 1.90810003742599)
idx: 4 -- playing baseline arm
(5, 3, 0.0)
(6, 1, 1.3478236409301474)
(7, 3, 0.0)
(8, 3, 0.0)
(9, 3, 0.0)
(10, 1, 1.8947556342444258)
(11, 3, 0.0)
(12, 2, 3.2635927047586382)
(13, 3, 0.0)
(14, 3, 0.0)
(15, 0, 2.6972202190987695)
idx: 15 -- playing baseline arm
(16, 3, 0.0)
(17, 2, 2.9944158658057232)
(18, 3, 0.0)
(19, 3, 0.0)
(20, 3, 0.0)
(21, 3, 0.0)
(22, 3, 0.0)
(23, 0, 0.66181075472808826)
idx: 23 -- playing baseline arm
(24, 3, 0.0)
(25, 3, 0.0)
(26, 1, 0.56401174978258017)
(27, 1, 3.1932265131229207)
(28, 3, 0.0)
(29, 3, 0.0)
(30, 3, 0.0)
(31, 0, 0.46814754041469131)
idx: 31 -- playing baseline arm
(32, 3, 0.0)
(33, 3, 0.0)
(34, 1, 0.70364341437910505)
(35, 3, 0.0)
(36, 3, 0.0)
(37, 3, 0.0)
(38, 0, 1.3923527540219902)
idx: 38 -- playing baseline arm
(39, 3, 0.0)
(40, 3, 0.0)
(41, 0, 1.3905238838827565)
idx: 41 -

(633, 3, 0.0)
(634, 3, 0.0)
(635, 3, 0.0)
(636, 3, 0.0)
(637, 3, 0.0)
(638, 3, 0.0)
(639, 1, 1.185563540430663)
(640, 3, 0.0)
(641, 3, 0.0)
(642, 3, 0.0)
(643, 3, 0.0)
(644, 3, 0.0)
(645, 3, 0.0)
(646, 3, 0.0)
(647, 3, 0.0)
(648, 0, 0.69514797034688869)
idx: 648 -- playing baseline arm
(649, 3, 0.0)
(650, 3, 0.0)
(651, 3, 0.0)
(652, 3, 0.0)
(653, 3, 0.0)
(654, 3, 0.0)
(655, 3, 0.0)
(656, 3, 0.0)
(657, 3, 0.0)
(658, 3, 0.0)
(659, 1, 0.77986862808849666)
(660, 3, 0.0)
(661, 3, 0.0)
(662, 3, 0.0)
(663, 3, 0.0)
(664, 3, 0.0)
(665, 3, 0.0)
(666, 3, 0.0)
(667, 3, 0.0)
(668, 3, 0.0)
(669, 3, 0.0)
(670, 3, 0.0)
(671, 3, 0.0)
(672, 1, 0.99805386889394088)
(673, 3, 0.0)
(674, 3, 0.0)
(675, 3, 0.0)
(676, 3, 0.0)
(677, 3, 0.0)
(678, 1, 0.181962971754695)
(679, 0, 0.56787635362856714)
idx: 679 -- playing baseline arm
(680, 3, 0.0)
(681, 3, 0.0)
(682, 0, 0.46221223721172433)
idx: 682 -- playing baseline arm
(683, 1, 0.6351360043483073)
(684, 3, 0.0)
(685, 3, 0.0)
(686, 3, 0.0)
(687, 1, 0.95669232981

(1251, 1, 0.92155725428294222)
(1252, 3, 0.0)
(1253, 3, 0.0)
(1254, 1, 0.76605450196400771)
(1255, 3, 0.0)
(1256, 3, 0.0)
(1257, 3, 0.0)
(1258, 3, 0.0)
(1259, 3, 0.0)
(1260, 3, 0.0)
(1261, 1, 0.55001901976393763)
(1262, 3, 0.0)
(1263, 3, 0.0)
(1264, 3, 0.0)
(1265, 3, 0.0)
(1266, 3, 0.0)
(1267, 3, 0.0)
(1268, 3, 0.0)
(1269, 3, 0.0)
(1270, 3, 0.0)
(1271, 3, 0.0)
(1272, 3, 0.0)
(1273, 3, 0.0)
(1274, 3, 0.0)
(1275, 3, 0.0)
(1276, 3, 0.0)
(1277, 3, 0.0)
(1278, 0, 0.62920634370499806)
idx: 1278 -- playing baseline arm
(1279, 3, 0.0)
(1280, 3, 0.0)
(1281, 3, 0.0)
(1282, 3, 0.0)
(1283, 2, 1.6383746397059382)
(1284, 3, 0.0)
(1285, 3, 0.0)
(1286, 3, 0.0)
(1287, 3, 0.0)
(1288, 3, 0.0)
(1289, 3, 0.0)
(1290, 0, 1.1701317357099956)
idx: 1290 -- playing baseline arm
(1291, 3, 0.0)
(1292, 3, 0.0)
(1293, 3, 0.0)
(1294, 3, 0.0)
(1295, 3, 0.0)
(1296, 3, 0.0)
(1297, 3, 0.0)
(1298, 3, 0.0)
(1299, 3, 0.0)
(1300, 3, 0.0)
(1301, 3, 0.0)
(1302, 3, 0.0)
(1303, 3, 0.0)
(1304, 3, 0.0)
(1305, 3, 0.0)
(1306, 3, 0.0

(1897, 3, 0.0)
(1898, 3, 0.0)
(1899, 3, 0.0)
(1900, 3, 0.0)
(1901, 3, 0.0)
(1902, 3, 0.0)
(1903, 1, 0.46846240226646618)
(1904, 3, 0.0)
(1905, 0, 0.73047724783262191)
idx: 1905 -- playing baseline arm
(1906, 3, 0.0)
(1907, 3, 0.0)
(1908, 3, 0.0)
(1909, 3, 0.0)
(1910, 0, 0.44034457853687548)
idx: 1910 -- playing baseline arm
(1911, 3, 0.0)
(1912, 3, 0.0)
(1913, 0, 0.78496041888361212)
idx: 1913 -- playing baseline arm
(1914, 3, 0.0)
(1915, 1, 0.75571894858956279)
(1916, 3, 0.0)
(1917, 3, 0.0)
(1918, 3, 0.0)
(1919, 3, 0.0)
(1920, 3, 0.0)
(1921, 3, 0.0)
(1922, 3, 0.0)
(1923, 3, 0.0)
(1924, 3, 0.0)
(1925, 3, 0.0)
(1926, 3, 0.0)
(1927, 0, 0.56768865032629401)
idx: 1927 -- playing baseline arm
(1928, 3, 0.0)
(1929, 3, 0.0)
(1930, 3, 0.0)
(1931, 3, 0.0)
(1932, 3, 0.0)
(1933, 3, 0.0)
(1934, 3, 0.0)
(1935, 3, 0.0)
(1936, 3, 0.0)
(1937, 3, 0.0)
(1938, 1, 0.51050315015387593)
(1939, 3, 0.0)
(1940, 3, 0.0)
(1941, 3, 0.0)
(1942, 3, 0.0)
(1943, 3, 0.0)
(1944, 3, 0.0)
(1945, 3, 0.0)
(1946, 3, 0.0)
(1

(2575, 3, 0.0)
(2576, 3, 0.0)
(2577, 3, 0.0)
(2578, 3, 0.0)
(2579, 3, 0.0)
(2580, 3, 0.0)
(2581, 3, 0.0)
(2582, 3, 0.0)
(2583, 3, 0.0)
(2584, 3, 0.0)
(2585, 3, 0.0)
(2586, 3, 0.0)
(2587, 1, 0.69869814647434825)
(2588, 3, 0.0)
(2589, 3, 0.0)
(2590, 3, 0.0)
(2591, 3, 0.0)
(2592, 3, 0.0)
(2593, 0, 0.93567713231034855)
idx: 2593 -- playing baseline arm
(2594, 3, 0.0)
(2595, 3, 0.0)
(2596, 3, 0.0)
(2597, 3, 0.0)
(2598, 3, 0.0)
(2599, 3, 0.0)
(2600, 3, 0.0)
(2601, 3, 0.0)
(2602, 3, 0.0)
(2603, 1, 0.59473867444765049)
(2604, 3, 0.0)
(2605, 3, 0.0)
(2606, 3, 0.0)
(2607, 3, 0.0)
(2608, 3, 0.0)
(2609, 3, 0.0)
(2610, 3, 0.0)
(2611, 3, 0.0)
(2612, 3, 0.0)
(2613, 3, 0.0)
(2614, 3, 0.0)
(2615, 3, 0.0)
(2616, 3, 0.0)
(2617, 3, 0.0)
(2618, 3, 0.0)
(2619, 3, 0.0)
(2620, 3, 0.0)
(2621, 3, 0.0)
(2622, 3, 0.0)
(2623, 3, 0.0)
(2624, 3, 0.0)
(2625, 3, 0.0)
(2626, 3, 0.0)
(2627, 3, 0.0)
(2628, 3, 0.0)
(2629, 3, 0.0)
(2630, 3, 0.0)
(2631, 3, 0.0)
(2632, 0, 1.122128824085364)
idx: 2632 -- playing baseline arm


...

(8006, 1, 0.0)
(8007, 1, 0.0)
(8008, 0, 0.04700964038891331)
idx: 8008 -- playing baseline arm
(8009, 0, 0.0)
idx: 8009 -- playing baseline arm
(8010, 1, 0.0)
(8011, 0, 0.0)
idx: 8011 -- playing baseline arm
(8012, 0, 0.0)
idx: 8012 -- playing baseline arm
(8013, 1, 0.0)
(8014, 0, 0.0)
idx: 8014 -- playing baseline arm
(8015, 1, 0.0)
(8016, 3, 0.31169757390001318)
(8017, 0, 0.0)
idx: 8017 -- playing baseline arm
(8018, 0, 0.019976863993032445)
idx: 8018 -- playing baseline arm
(8019, 0, 0.045286748187516435)
idx: 8019 -- playing baseline arm
(8020, 0, 0.0)
idx: 8020 -- playing baseline arm
(8021, 1, 0.0)
(8022, 1, 0.0)
(8023, 0, 0.0)
idx: 8023 -- playing baseline arm
(8024, 1, 0.0)
(8025, 1, 0.0)
(8026, 1, 0.0)
(8027, 0, 0.0)
idx: 8027 -- playing baseline arm
(8028, 1, 0.0)
(8029, 1, 0.0)
(8030, 1, 0.0)
(8031, 2, 0.24051280440045725)
(8032, 1, 0.0)
(8033, 1, 0.0)
(8034, 0, 0.0)
idx: 8034 -- playing baseline arm
(8035, 3, 0.7555564406290789)
(8036, 0, 0.0)
idx: 8036 -- playing baseline 

(8546, 0, 0.0)
idx: 8546 -- playing baseline arm
(8547, 1, 0.0)
(8548, 1, 0.0)
(8549, 0, 0.0)
idx: 8549 -- playing baseline arm
(8550, 0, 0.0)
idx: 8550 -- playing baseline arm
(8551, 0, 0.0)
idx: 8551 -- playing baseline arm
(8552, 0, 0.0)
idx: 8552 -- playing baseline arm
(8553, 0, 0.0)
idx: 8553 -- playing baseline arm
(8554, 1, 0.0)
(8555, 0, 0.0079886269191540293)
idx: 8555 -- playing baseline arm
(8556, 0, 0.0)
idx: 8556 -- playing baseline arm
(8557, 1, 0.0)
(8558, 1, 0.0)
(8559, 2, 0.40360762627685909)
(8560, 1, 0.0)
(8561, 1, 0.0)
(8562, 1, 0.0)
(8563, 0, 0.0)
idx: 8563 -- playing baseline arm
(8564, 0, 0.0)
idx: 8564 -- playing baseline arm
(8565, 1, 0.0)
(8566, 2, 0.41649801326880803)
(8567, 1, 0.0)
(8568, 1, 0.0)
(8569, 1, 0.0)
(8570, 0, 0.0)
idx: 8570 -- playing baseline arm
(8571, 0, 0.0)
idx: 8571 -- playing baseline arm
(8572, 0, 0.0)
idx: 8572 -- playing baseline arm
(8573, 1, 0.0)
(8574, 1, 0.0)
(8575, 1, 0.0)
(8576, 1, 0.0)
(8577, 1, 0.0)
(8578, 1, 0.0)
(8579, 1, 0.0

(8851, 1, 0.0)
(8852, 0, 0.0)
idx: 8852 -- playing baseline arm
(8853, 1, 0.0)
(8854, 1, 0.0)
(8855, 2, 0.51096726110504043)
(8856, 1, 0.0)
(8857, 1, 0.0)
(8858, 1, 0.0)
(8859, 0, 0.0)
idx: 8859 -- playing baseline arm
(8860, 1, 0.0)
(8861, 1, 0.0)
(8862, 0, 0.0)
idx: 8862 -- playing baseline arm
(8863, 1, 0.0)
(8864, 1, 0.0)
(8865, 1, 0.0)
(8866, 1, 0.0)
(8867, 0, 0.0)
idx: 8867 -- playing baseline arm
(8868, 1, 0.0)
(8869, 1, 0.0)
(8870, 1, 0.0)
(8871, 1, 0.0)
(8872, 0, 0.0)
idx: 8872 -- playing baseline arm
(8873, 1, 0.0)
(8874, 1, 0.0)
(8875, 1, 0.0)
(8876, 2, 0.070396966432282521)
(8877, 1, 0.0)
(8878, 1, 0.0)
(8879, 1, 0.0)
(8880, 1, 0.0)
(8881, 1, 0.0)
(8882, 0, 0.0028556433498259093)
idx: 8882 -- playing baseline arm
(8883, 0, 0.0)
idx: 8883 -- playing baseline arm
(8884, 1, 0.0)
(8885, 1, 0.0)
(8886, 0, 0.0)
idx: 8886 -- playing baseline arm
(8887, 1, 0.0)
(8888, 0, 0.0)
idx: 8888 -- playing baseline arm
(8889, 1, 0.0)
(8890, 1, 0.0)
(8891, 1, 0.0)
(8892, 1, 0.0)
(8893, 0, 0.0

(9178, 1, 0.0)
(9179, 1, 0.0)
(9180, 0, 0.0)
idx: 9180 -- playing baseline arm
(9181, 0, 0.0)
idx: 9181 -- playing baseline arm
(9182, 0, 0.0)
idx: 9182 -- playing baseline arm
(9183, 0, 0.10108101571034034)
idx: 9183 -- playing baseline arm
(9184, 0, 0.0074741102106876678)
idx: 9184 -- playing baseline arm
(9185, 1, 0.0)
(9186, 1, 0.0)
(9187, 1, 0.0)
(9188, 1, 0.0)
(9189, 1, 0.0)
(9190, 1, 0.0)
(9191, 2, 0.46002959660697695)
(9192, 1, 0.0)
(9193, 0, 0.0)
idx: 9193 -- playing baseline arm
(9194, 1, 0.0)
(9195, 1, 0.0)
(9196, 1, 0.0)
(9197, 1, 0.0)
(9198, 1, 0.0)
(9199, 0, 0.0)
idx: 9199 -- playing baseline arm
(9200, 0, 0.072783457344239189)
idx: 9200 -- playing baseline arm
(9201, 0, 0.050645249731443354)
idx: 9201 -- playing baseline arm
(9202, 1, 0.0)
(9203, 1, 0.0)
(9204, 1, 0.0)
(9205, 1, 0.0)
(9206, 1, 0.0)
(9207, 0, 0.0)
idx: 9207 -- playing baseline arm
(9208, 0, 0.0)
idx: 9208 -- playing baseline arm
(9209, 1, 0.0)
(9210, 1, 0.0)
(9211, 0, 0.0)
idx: 9211 -- playing baseline ar

(9726, 0, 0.016891580204474455)
idx: 9726 -- playing baseline arm
(9727, 1, 0.0)
(9728, 1, 0.0)
(9729, 0, 0.0)
idx: 9729 -- playing baseline arm
(9730, 0, 0.0)
idx: 9730 -- playing baseline arm
(9731, 1, 0.0)
(9732, 1, 0.0)
(9733, 1, 0.0)
(9734, 0, 0.0)
idx: 9734 -- playing baseline arm
(9735, 0, 0.0)
idx: 9735 -- playing baseline arm
(9736, 0, 0.0)
idx: 9736 -- playing baseline arm
(9737, 1, 0.0)
(9738, 0, 0.0)
idx: 9738 -- playing baseline arm
(9739, 0, 0.0)
idx: 9739 -- playing baseline arm
(9740, 0, 0.0)
idx: 9740 -- playing baseline arm
(9741, 1, 0.0)
(9742, 1, 0.0)
(9743, 1, 0.0)
(9744, 2, 0.33856907788526219)
(9745, 1, 0.0)
(9746, 0, 0.0)
idx: 9746 -- playing baseline arm
(9747, 1, 0.0)
(9748, 1, 0.0)
(9749, 1, 0.0)
(9750, 1, 0.0)
(9751, 1, 0.0)
(9752, 0, 0.0)
idx: 9752 -- playing baseline arm
(9753, 1, 0.0)
(9754, 0, 0.0)
idx: 9754 -- playing baseline arm
(9755, 1, 0.0)
(9756, 1, 0.0)
(9757, 1, 0.0)
(9758, 1, 0.0)
(9759, 1, 0.0)
(9760, 1, 0.0)
(9761, 0, 0.0)
idx: 9761 -- playin

## Threshold cons for comparison

In [8]:
BanditAlg = ThresholdConsBandit    
regret_thrcons = np.zeros((M, N))
arm_pulls_thrcons = np.zeros((M, N, k+1))
expt_rewards_thrcons = np.zeros((M, N))
expt_rewards_baseline_thrcons = np.zeros((M, N))

In [9]:
for m in range(M):
    params = LinearGeneratorParams(np.atleast_2d(alphas[m,:,:]), betas[m,:], d = d, k = k, intercept = intercept)
    generator = LinearGenerator(params)
    means = expected_regret_per_arm(generator)
    #Choose the baseline arm as the worst arm....
    sorted_means = np.sort(means)
    #m_idx = np.argmax(means)
    m_idx = np.where(means == sorted_means[baseline_idx])[0][0]
    base_alpha = alphas[m,m_idx,:]
    base_beta = betas[m,m_idx]
    bandit = BanditAlg(generator)
    print("Run: %d/%d"%(m+1,M))
    for i in range(N):
        (ctx, arm_idx, obs, r) = bandit.step()
        regret_thrcons[m,i] = r
        if arm_idx >= 0:
            arm_pulls_linucb[m,i,arm_idx] = 1
        else:
            arm_pulls_linucb[m,i,k] = 1
        expt_rewards_baseline_thrcons[m,i] = np.dot(ctx[1:],base_alpha) + base_beta
        expt_rewards_thrcons[m,i] = np.dot(ctx[1:], alphas[m,arm_idx,:]) + betas[m,arm_idx]


Run: 1/3
Run: 2/3
Run: 3/3


## Threshold greedy

In [ ]:
BanditAlg = ThresholdBandit    
regret_thrgre = np.zeros((M, N))
arm_pulls_thrgre = np.zeros((M, N, k+1))
expt_rewards_thrgre = np.zeros((M, N))
expt_rewards_baseline_thrgre = np.zeros((M, N))

In [ ]:
for m in range(M):
    params = LinearGeneratorParams(np.atleast_2d(alphas[m,:,:]), betas[m,:], d = d, k = k, intercept = intercept)
    generator = LinearGenerator(params)
    means = expected_regret_per_arm(generator)
    #Choose the baseline arm as the worst arm....
    sorted_means = np.sort(means)
    #m_idx = np.argmax(means)
    m_idx = np.where(means == sorted_means[baseline_idx])[0][0]
    base_alpha = alphas[m,m_idx,:]
    base_beta = betas[m,m_idx]
    bandit = BanditAlg(generator)
    print("Run: %d/%d"%(m+1,M))
    for i in range(N):
        (ctx, arm_idx, obs, r) = bandit.step()
        regret_thrgre[m,i] = r
        if arm_idx >= 0:
            arm_pulls_linucb[m,i,arm_idx] = 1
        else:
            arm_pulls_linucb[m,i,k] = 1
        expt_rewards_baseline_thrgre[m,i] = np.dot(ctx[1:],base_alpha) + base_beta
        expt_rewards_thrgre[m,i] = np.dot(ctx[1:], alphas[m,arm_idx,:]) + betas[m,arm_idx]


Run: 1/3
Run: 2/3


## Comparison of the methods

Trade-off between convergence rate, risk, and confidence level

In [ ]:
#Vary the confidence level, plot convergence rate, regret, risk
sns.tsplot(np.cumsum(regret, axis = 1), color = 'red')
sns.tsplot(np.cumsum(regret_linucb, axis = 1), color = 'blue')
sns.tsplot(np.cumsum(regret_thrgre, axis = 1), color = 'green')
sns.tsplot(np.cumsum(regret_thrcons, axis = 1), color = 'black')
plt.xlabel('round')
plt.ylabel('cumulative regret')
plt.legend(['CLUCB', 'LinUCB', 'Threshold greedy', 'Threshold cons'])
sns.despine(trim = True)

In [ ]:
#Vary the confidence level, plot convergence rate, regret, risk
sns.tsplot(np.divide(np.cumsum(regret, axis = 1), np.arange(N)), color = 'red')
sns.tsplot(np.divide(np.cumsum(regret_linucb, axis = 1),np.arange(N)), color = 'blue')
plt.xlabel('round')
plt.ylabel('per step regret')
plt.legend(['CLUCB', 'LinUCB'])
sns.despine(trim = True)

## Plot the violation of constraints........

In [ ]:
#cum_expt_rewards_rs = np.cumsum(expt_rewards_rs, axis = 1)
#cum_expt_rewards_baseline_rs = np.cumsum(expt_rewards_baseline_rs, axis = 1)
cum_expt_rewards_ucb = np.cumsum(expt_rewards_linucb, axis = 1)
cum_expt_rewards_baseline_ucb = np.cumsum(expt_rewards_baseline_linucb, axis = 1)
cum_expt_rewards_thr = np.cumsum(expt_rewards_thrgre, axis = 1)
cum_expt_rewards_baseline_thr = np.cumsum(expt_rewards_baseline_thrgre, axis = 1)
cum_expt_rewards_thrcons = np.cumsum(expt_rewards_thrcons, axis = 1)
cum_expt_rewards_baseline_thrcons = np.cumsum(expt_rewards_baseline_thrcons, axis = 1)
#cum_expt_rewards_gre = np.cumsum(expt_rewards_gre, axis = 1)
#cum_expt_rewards_baseline_gre = np.cumsum(expt_rewards_baseline_gre, axis = 1)
cum_expt_rewards_clucb = np.cumsum(expt_rewards_clucb, axis = 1)
cum_expt_rewards_baseline_clucb = np.cumsum(expt_rewards_baseline_clucb, axis = 1)

constraint_violation_clucb = np.mean((cum_expt_rewards_clucb.T - (1-alpha)*cum_expt_rewards_baseline_clucb.T)<0,0)
#constraint_violation_gre = np.mean((cum_expt_rewards_gre.T - (1-alpha_clucb)*cum_expt_rewards_baseline_gre.T)<0,0)
constraint_violation_ucb = np.mean((cum_expt_rewards_ucb.T - (1-alpha)*cum_expt_rewards_baseline_ucb.T)<0,0)
#constraint_violation_rs = np.mean((cum_expt_rewards_rs.T - (1-alpha_clucb)*cum_expt_rewards_baseline_rs.T)<0,0)
constraint_violation_thr = np.mean((cum_expt_rewards_thr.T - (1-alpha)*cum_expt_rewards_baseline_thr.T)<0,0)
constraint_violation_thrcons = np.mean((cum_expt_rewards_thrcons.T - (1-alpha)*cum_expt_rewards_baseline_thrcons.T)<0,0)

all_method_constraints = 100*np.stack((constraint_violation_ucb,
                                       constraint_violation_thr,
                                       constraint_violation_thrcons,
                                       constraint_violation_clucb))

#all_method_constraints = 100*np.stack((constraint_violation_ucb,
#                                       constraint_violation_gre,
#                                       constraint_violation_thr,
#                                       constraint_violation_thrcons,
#                                       constraint_violation_rs,
#                                       constraint_violation_clucb))

#df_constraints = pd.DataFrame(data = all_method_constraints.T, columns = ['LinUCB', 'Greedy', 'RS-greedy', 'RS-conservative', 'RS-LinUCB', 'CLUCB'], index = range(M))
df_constraints = pd.DataFrame(data = all_method_constraints.T, columns = ['LinUCB', 'RS-greedy', 'RS-conservative', 'CLUCB'], index = range(M))

In [ ]:
sns.barplot(data = df_constraints)
plt.xlabel("Method")
plt.ylabel("Percentage time violating baseline performance constraint")
sns.despine(trim = True)

In [ ]:
#cum_expt_rewards_clucb = np.cumsum(expt_rewards_clucb, axis = 1)
#cum_expt_rewards_baseline_clucb = np.cumsum(expt_rewards_baseline_clucb, axis = 1)

#plt.plot(cum_expt_rewards_clucb.T)
#cum_expt_rewards_clucb.shape
plt.plot(cum_expt_rewards_clucb.T - (1-alpha)*cum_expt_rewards_baseline_clucb.T);